In [1]:
from analyze_csv import *
from mutations import get_all_moves
from evaluation_indices import *
from batch_tasks import *
from data_generation import *
import numpy as np
import pandas as pd
import os
import traceback
import pickle
from sklearn.decomposition import PCA
from scipy.stats import entropy, skew, kurtosis
from autosklearn.classification import AutoSklearnClassifier

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
all_moves = np.array(get_all_moves())
all_moves

array(['unguided_merge_gene_move', 'unguided_remove_and_reclassify_move',
       'unguided_split_gene_move', 'expand_cluster_move',
       'split_farthest_move', 'unguided_eliminate_move',
       'knn_reclassification_move', 'one_nth_change_move',
       'centroid_k_means_move', 'centroid_hill_climbing_move',
       'prototype_k_means_move', 'prototype_hill_climbing_move',
       'tree_hill_climbing_move',
       'guided_remove_and_reclassify_move("density_based_validity_separation")',
       'guided_remove_and_reclassify_move("density_based_sparseness_separation")',
       'guided_remove_and_reclassify_move("mean_centroid_distance_separation")',
       'guided_remove_and_reclassify_move("diameter_separation")',
       'guided_histogram_split_gene_move("density_based_validity_separation")',
       'guided_histogram_split_gene_move("density_based_sparseness_separation")',
       'guided_histogram_split_gene_move("mean_centroid_distance_separation")',
       'guided_histogram_split_gene_

In [3]:
index_minimized = {}
for name, func in indices:
    index_minimized[name] = eval(func).is_minimized
index_minimized

{'silhouette': False,
 'calinski_harabaz': False,
 'davies_bouldin': True,
 'dvcb': False,
 'dunn': False,
 'generalized_dunn_41': False,
 'generalized_dunn_43': False,
 'generalized_dunn_51': False,
 'generalized_dunn_53': False,
 'generalized_dunn_13': False}

In [4]:
if False:
    mutation_usage = []
    for idx, filename in enumerate(os.listdir('/tmp/results-1010/')):
        index_name = filename[:filename.find('-')]
        try:
            data, time = read_results('/tmp/results-1010/' + filename)
        except:
            traceback.print_exc()
            continue
        n_clusters = data.iloc[-1]['n_clusters']
        unique_numbers = np.unique(np.bincount(data.index[1:]))
        mutation_names = np.array([s[:s.find(':')] if s is not None and s != 'None' else s for s in data['detail']])
        mut_indices = np.argwhere(mutation_names[:, None] == all_moves[None, :])[:, 1]
        assert len(mut_indices) == len(data) - 1
        data['mutation_index'] = np.hstack([[-1], mut_indices])
        index_series = [t['index'].rename(mutation_names[i] + '_index') for i, t in data[['mutation_index', 'index']].groupby('mutation_index')]
        table = pd.concat(index_series, axis=1).values
        if not index_minimized[index_name]:
            max_indices = np.nanargmax(table, axis=1)
            max_values = np.nanmax(table, axis=1)
            change_steps = np.argwhere(max_values[1:] > max_values[:-1]).flatten() + 1
        else:
            max_indices = np.nanargmin(table, axis=1)
            max_values = np.nanmin(table, axis=1)
            change_steps = np.argwhere(max_values[1:] < max_values[:-1]).flatten() + 1
        c_usage = np.unique(max_indices[change_steps]) - 1
        mutation_usage.append((filename, c_usage))
        print(idx, filename, n_clusters, len(unique_numbers), len(c_usage), len(change_steps))
    with open('mutation_usage.dat', 'wb') as f:
        pickle.dump(mutation_usage, f)
else:
    with open('mutation_usage.dat', 'rb') as f:
        mutation_usage = pickle.load(f)

In [5]:
np.bincount(np.hstack([a[1] for  a in mutation_usage]))

array([318, 398, 191, 314, 177, 353, 189, 258, 323, 151, 308, 257, 279,
       282, 278, 252, 156,  90, 109, 160, 168, 146, 159, 232, 219, 236,
       199, 339, 358,  78,  32])

In [6]:
generated_prefix = 'scratch'
real_prefix = 'datasets'
tasks_list = init_batch(real_prefix)
datasDict = {a: {'dataset': eval(b)} for a, b in datas}
datasDict.keys()

Excluding column number 2 because it is a single value
Excluding column number 23 because it is a single value


dict_keys(['generated_2dim_10cl', 'generated_2dim_30cl', 'generated_10dim_10cl', 'generated_10dim_30cl', 'immunotherapy', 'user_knowledge', 'sales_transactions', 'Concrete_Data', 'HappinessRank_2015', 'ICU', 'ParkinsonSpeechDataset', 'analcatdata_lawsuit', 'analcatdata_vineyard', 'analcatdata_wildcat', 'arsenic_female_bladder', 'arsenic_male_bladder', 'arsenic_male_lung', 'autoUniv_au7_1100', 'autoUniv_au7_500', 'autoUniv_au7_700', 'badges2', 'banknote_authentication', 'boston_corrected', 'breast_tissue', 'chscase_census2', 'chscase_census6', 'collins', 'dataset_185_yeast', 'dataset_187_abalone', 'dataset_18_mfeat_morphological', 'dataset_2181_auto_price', 'dataset_36_segment', 'dataset_39_ecoli', 'dataset_41_glass', 'dataset_43_haberman', 'dataset_53_heart_statlog', 'dataset_54_vehicle', 'dataset_8_liver_disorders', 'debutanizer', 'disclosure_x_bias', 'fri_c0_1000_25', 'fri_c0_100_25', 'fri_c0_250_10', 'fri_c1_1000_5', 'fri_c1_100_10', 'fri_c1_100_5', 'fri_c1_250_10', 'fri_c1_250_5', 

In [7]:
for k, v in datasDict.items():
    dataset = v['dataset']
    v['ma1'] = np.log2(len(dataset))
    v['ma2'] = np.log2(dataset.shape[1])
    meta_attrs = []
    meta_entropy = []
    cont_attrs = []
    for i, s in enumerate(dataset.T):
        uniq = np.unique(s)
        uniq.sort()
        if len(uniq) < len(s) * 0.3:
            meta_attrs.append(i)
            ints = np.searchsorted(uniq, s)
            meta_entropy.append(entropy(ints))
        else:
            cont_attrs.append(i)
    v['ma3'] = len(meta_attrs)
    v['ma5'] = sum(meta_entropy) / len(meta_entropy) if len(meta_entropy) > 0 else 0
    # TODO ma6
    if len(cont_attrs) > 0:
        corr = np.abs(np.corrcoef(dataset[:, cont_attrs], rowvar=False))
        v['ma7'] = corr[np.arange(0, len(corr))[None, :] <= np.arange(0, len(corr))[:, None]].mean() if isinstance(corr, np.ndarray) else corr
        v['ma8'] = np.mean(skew(dataset[:, cont_attrs]))
        v['ma9'] = np.mean(kurtosis(dataset[:, cont_attrs]))
    else:
        v['ma7'], v['ma8'], v['ma9'] = 0, 0, 0
    dists = pdist(dataset)
    v['md1'] = dists.mean()
    v['md2'] = dists.var()
    v['md3'] = dists.std()
    v['md4'] = skew(dists)
    v['md5'] = kurtosis(dists)
    hist = np.histogram(dists, bins=9, range=(0, dists.max()))[0]
    assert len(hist) == 9
    for i, h in enumerate(hist):
        v['md' + str(i + 6)] = h
    z = np.abs(((dists - v['md1']) / v['md3']))
    v['md16'] = (z < 1).sum()
    v['md17'] = ((1 <= z) & (z < 2)).sum()
    v['md18'] = ((2 <= z) & (z < 3)).sum()
    v['md19'] = (3 <= z).sum()

In [40]:
meta_knowledge_list = {}
def get_meta_list(name):
    if name not in meta_knowledge_list:
        meta_knowledge_list[name] = []
    return meta_knowledge_list[name]

In [41]:
for fname, mutations in mutation_usage:
    index, data = fname.split('-')[:2]
    get_meta_list('index').append(index)
    metadata = datasDict[data]
    for k, v in metadata.items():
        if k == 'dataset': continue
        get_meta_list(k).append(v)
    arr = np.zeros(len(all_moves), dtype='bool')
    arr[mutations] = True
    for i, v in enumerate(arr):
        get_meta_list('rm' + str(i)).append(v)
assert len(set(map(len, meta_knowledge_list.values()))) == 1

In [42]:
metaframe = pd.DataFrame(meta_knowledge_list)
metaframe

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,calinski_harabaz,9.972980,3.169925,7,6.470232,0.728816,0.071564,-0.210538,0.066450,0.000840,...,False,False,False,True,False,True,True,True,False,False
1,calinski_harabaz,7.303781,3.584963,1,4.865607,0.411555,0.160897,0.462070,0.156211,0.002364,...,False,False,False,False,True,False,False,False,False,False
2,calinski_harabaz,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,False,False,False,False,False,False,False,False,False,False
3,calinski_harabaz,10.020980,4.857981,6,6.421704,0.444560,1.585362,6.328814,0.122503,0.002991,...,False,False,False,True,False,False,True,False,False,False
4,calinski_harabaz,8.038919,2.321928,4,4.612358,1.000000,0.952002,0.728251,0.102716,0.004888,...,False,False,False,False,False,True,False,False,False,False
5,calinski_harabaz,8.870365,2.000000,4,5.930552,0.000000,0.000000,0.000000,0.049335,0.000641,...,False,False,False,False,False,False,True,False,False,False
6,calinski_harabaz,7.219169,2.584963,3,4.379609,0.846739,1.049661,3.314253,0.167310,0.007739,...,True,False,False,True,False,False,False,False,False,False
7,calinski_harabaz,9.126704,2.321928,4,5.563692,1.000000,9.651291,105.838133,0.060682,0.006092,...,False,False,False,False,False,False,False,False,True,False
8,calinski_harabaz,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,0.006126,...,False,False,False,True,True,False,True,False,False,False
9,calinski_harabaz,9.126704,2.321928,4,5.650277,1.000000,9.600139,104.073434,0.060185,0.006170,...,False,False,False,False,False,True,False,True,False,False


In [43]:
indices_list = np.array(list(index_minimized))
metaframe['index'] = np.argwhere(metaframe['index'].values[:, None] == indices_list[None, :])[:, 1].flatten()
metaframe

,index,ma1,ma2,ma3,ma5,ma7,ma8,ma9,md1,md2,...,rm21,rm22,rm23,rm24,rm25,rm26,rm27,rm28,rm29,rm30
0,1,9.972980,3.169925,7,6.470232,0.728816,0.071564,-0.210538,0.066450,0.000840,...,False,False,False,True,False,True,True,True,False,False
1,1,7.303781,3.584963,1,4.865607,0.411555,0.160897,0.462070,0.156211,0.002364,...,False,False,False,False,True,False,False,False,False,False
2,1,7.643856,4.321928,17,3.287976,0.522758,0.023865,0.174592,0.142564,0.002888,...,False,False,False,False,False,False,False,False,False,False
3,1,10.020980,4.857981,6,6.421704,0.444560,1.585362,6.328814,0.122503,0.002991,...,False,False,False,True,False,False,True,False,False,False
4,1,8.038919,2.321928,4,4.612358,1.000000,0.952002,0.728251,0.102716,0.004888,...,False,False,False,False,False,True,False,False,False,False
5,1,8.870365,2.000000,4,5.930552,0.000000,0.000000,0.000000,0.049335,0.000641,...,False,False,False,False,False,False,True,False,False,False
6,1,7.219169,2.584963,3,4.379609,0.846739,1.049661,3.314253,0.167310,0.007739,...,True,False,False,True,False,False,False,False,False,False
7,1,9.126704,2.321928,4,5.563692,1.000000,9.651291,105.838133,0.060682,0.006092,...,False,False,False,False,False,False,False,False,True,False
8,1,9.126704,2.321928,4,5.578625,1.000000,9.600139,104.073434,0.060484,0.006126,...,False,False,False,True,True,False,True,False,False,False
9,1,9.126704,2.321928,4,5.650277,1.000000,9.600139,104.073434,0.060185,0.006170,...,False,False,False,False,False,True,False,True,False,False


In [55]:
metaframe.to_csv('metaframe.csv')

In [47]:
train_columns = [c for c in metaframe if not c.startswith('rm')]
test_columns = [c for c in metaframe if c.startswith('rm')]
train_columns

['index',
 'ma1',
 'ma2',
 'ma3',
 'ma5',
 'ma7',
 'ma8',
 'ma9',
 'md1',
 'md2',
 'md3',
 'md4',
 'md5',
 'md6',
 'md7',
 'md8',
 'md9',
 'md10',
 'md11',
 'md12',
 'md13',
 'md14',
 'md16',
 'md17',
 'md18',
 'md19']

In [45]:
from sklearn.cross_validation import train_test_split
meta_train, meta_test, res_train, res_test = train_test_split(metaframe[train_columns], metaframe[test_columns])

In [54]:
cls = AutoSklearnClassifier(n_jobs=4)
cls.fit(meta_train, res_train, feat_type=['Categorical'] + ['Numerical'] * (len(train_columns) - 1))

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-package

[WARNING] [2019-05-08 13:47:17,506:EnsembleBuilder(1791095845):877daf3392de26b6b8f2bab9f4c4a56f] No models better than random - using Dummy Score!
[WARNING] [2019-05-08 13:47:17,562:EnsembleBuilder(1791095845):877daf3392de26b6b8f2bab9f4c4a56f] No models better than random - using Dummy Score!


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384

[WARNING] [2019-05-08 13:47:32,332:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-05-08 13:47:32,332:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384

/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384kfgxm3iv1-python3-3.6.8-env/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/nix/store/cfr65vfb7crcqb11l1fi384

KeyboardInterrupt: 

In [76]:
mut_flags = []
for r in mut_indices:
    flags = np.zeros(len(mutation_names), dtype='bool')
    flags[r] = True
    mut_flags.append(flags)
mut_flags = np.array(mut_flags)

In [85]:
for fname, _ in mutation_usage:
    index, data = fname.split('-')[:2]
    dataset = datasDict[data]
    normalized = PCA(n_components=dataset.shape[1]).fit_transform(dataset)
    percentile = np.percentile(normalized, [25, 75], axis=0)
    percentDiff = percentile[1] - percentile[0]
    percentile[0] -= percentDiff * 1.5
    percentile[1] += percentDiff * 1.5
    print(percentile)
    get_meta_list('ma4').append(((normalized < percentile[0]) | (normalized > percentile[1])).any().sum())

array([71, 37, 45, 43, 36, 29, 23, 21, 29, 32, 37, 26, 20, 25, 28, 30, 19,
       26, 55, 32, 24, 18, 30, 12])